In [1]:
import shutil
import numpy as np
import pandas as pd
import re

In [2]:
nonLa_image_metadata = pd.read_csv('../../data_samples/results/processed_subset_results/local_nonLa_art.csv')

### TODO: Fix Current Issue / Duplicate image names and authors are in the metadata, and during download I was only able to reach one, as the duplicates were all overwritten. (one name/many images) Complete
### TODO: Replace naming convention for file names by appending the objectID tag which will fully resolve issue with duplicate rows. There may also be a sampling issue during the creation of the nonLA image sample which caused the data to be sampled with replacement which potentially caused so many redundant rows and images during downloading which increased the load time. Complete

In [ ]:
# code to add percent from artist column in non-la art to match la dataset
#proportion_ofArtist = nonla_image_metadata.forwarddisplayname.value_counts(normalize=True)
#nonla_image_metadata['percent_fromArtist'] = nonla_image_metadata.forwarddisplayname.apply(lambda x: proportion_ofArtist[x])

In [3]:
nonLa_image_metadata.columns

Index(['objectid', 'title', 'medium', 'dimensions', 'inscription', 'markings',
       'attributioninverted', 'attribution', 'provenancetext', 'creditline',
       'classification', 'parentid', 'isvirtual', 'portfolio', 'series',
       'sequence', 'width', 'height', 'maxpixels', 'depictstmsobjectid',
       'assistivetext', 'constituentid', 'displayorder', 'ulanid',
       'preferreddisplayname', 'forwarddisplayname', 'lastname', 'nationality',
       'constituenttype', 'expanded_url', 'percent_fromCountry',
       'Country Name', 'Country', 'Continent', 'file_name', 'directory',
       'file_downloaded'],
      dtype='object')

### Identifying the duplicates which causes a reduction in the num of images downloaded due to file name overwriting issue (duplicates have same name, author, and objectid)

In [21]:
nonLa_image_metadata.columns

Index(['objectid', 'title', 'medium', 'dimensions', 'inscription', 'markings',
       'attributioninverted', 'attribution', 'provenancetext', 'creditline',
       'classification', 'parentid', 'isvirtual', 'portfolio', 'series',
       'sequence', 'width', 'height', 'maxpixels', 'depictstmsobjectid',
       'assistivetext', 'constituentid', 'displayorder', 'ulanid',
       'preferreddisplayname', 'forwarddisplayname', 'lastname', 'nationality',
       'constituenttype', 'expanded_url', 'percent_fromCountry',
       'Country Name', 'Country', 'Continent', 'file_name', 'directory',
       'file_downloaded'],
      dtype='object')

In [22]:
#these are files we need to reinspect due to their multiplicity
series_num_objs = nonLa_image_metadata.loc[:,['title', 'lastname', 'preferreddisplayname', 'objectid']].groupby(['title', 'lastname', 'objectid']).apply(len)

In [23]:
series = series_num_objs[series_num_objs > 2]

In [24]:
series_groups = series.index

In [26]:
series_groups[0]

('"Ah Mr. Hardy, Mr. Hardy, if you only knew all the circumstances..."',
 'Dyson',
 6829.0)

In [27]:
duplicates = None
for title, author, objectid in series_groups:
    dupe_partition = nonLa_image_metadata.where((nonLa_image_metadata.title == title) & (nonLa_image_metadata.lastname == author) & (nonLa_image_metadata.objectid == objectid)).dropna(how='all')
    if duplicates is None:
        duplicates = dupe_partition
    else:
        duplicates = pd.concat([duplicates, dupe_partition])

In [28]:
duplicates = duplicates.reset_index(drop=False)

In [126]:
duplicates.head()

,index,objectid,title,medium,dimensions,inscription,markings,attributioninverted,attribution,provenancetext,...,nationality,constituenttype,expanded_url,percent_fromCountry,Country Name,Country,Continent,file_name,directory,file_downloaded
0,184,6829.0,"""Ah Mr. Hardy, Mr. Hardy, if you only knew all...",drypoint,NaN,NaN,NaN,"Dyson, Will",Will Dyson,NaN,...,Australian,individual,https://api.nga.gov/iiif/9b625788-593a-4e9a-9c...,0.000356,Australia,AU,Oceania,"""Ah_Mr._Hardy,_Mr._Hardy,_if_you_only_knew_all...",../../non_laImages,True
1,721,6829.0,"""Ah Mr. Hardy, Mr. Hardy, if you only knew all...",drypoint,NaN,NaN,NaN,"Dyson, Will",Will Dyson,NaN,...,Australian,individual,https://api.nga.gov/iiif/9b625788-593a-4e9a-9c...,0.000356,Australia,AU,Oceania,"""Ah_Mr._Hardy,_Mr._Hardy,_if_you_only_knew_all...",../../non_laImages,True
2,1660,6829.0,"""Ah Mr. Hardy, Mr. Hardy, if you only knew all...",drypoint,NaN,NaN,NaN,"Dyson, Will",Will Dyson,NaN,...,Australian,individual,https://api.nga.gov/iiif/9b625788-593a-4e9a-9c...,0.000356,Australia,AU,Oceania,"""Ah_Mr._Hardy,_Mr._Hardy,_if_you_only_knew_all...",../../non_laImages,True
3,1951,6829.0,"""Ah Mr. Hardy, Mr. Hardy, if you only knew all...",drypoint,NaN,NaN,NaN,"Dyson, Will",Will Dyson,NaN,...,Australian,individual,https://api.nga.gov/iiif/9b625788-593a-4e9a-9c...,0.000356,Australia,AU,Oceania,"""Ah_Mr._Hardy,_Mr._Hardy,_if_you_only_knew_all...",../../non_laImages,True
4,357,6827.0,"""Can you forgive me Father?"" - ""Can you forgiv...",drypoint,NaN,NaN,NaN,"Dyson, Will",Will Dyson,NaN,...,Australian,individual,https://api.nga.gov/iiif/1446efef-983e-4ea6-8c...,0.000356,Australia,AU,Oceania,"""Can_you_forgive_me_Father?""_-_""Can_you_forgiv...",../../non_laImages,True


In [30]:
unique_from_duplicates = duplicates.groupby(['title', 'lastname', 'objectid'])

In [76]:
unique = None
to_trash = None

In [71]:
nonLa_image_metadata.shape

(1950, 37)

In [77]:
for art_info, data in unique_from_duplicates:
    single_pick = data.iloc[0,:]
    trash = data.loc[1:, 'index']
    #collect all the unique datas into unique df for future download
    if unique is None:
        unique = pd.DataFrame(single_pick)
    else:
        unique = pd.concat([unique, single_pick], axis=1)
    if to_trash is None:
        to_trash = trash
    else:
        to_trash = pd.concat([to_trash, trash])
    #remove all duplicates (except for one) from the nonLa image metadata set
    #nonLa_image_metadata = nonLa_image_metadata.drop(to_trash)

In [78]:
to_keep = unique.T

In [79]:
to_keep = to_keep.reset_index(drop=True)

In [127]:
to_keep.head()

,index,objectid,title,medium,dimensions,inscription,markings,attributioninverted,attribution,provenancetext,...,nationality,constituenttype,expanded_url,percent_fromCountry,Country Name,Country,Continent,file_name,directory,file_downloaded
0,184,6829.0,"""Ah Mr. Hardy, Mr. Hardy, if you only knew all...",drypoint,NaN,NaN,NaN,"Dyson, Will",Will Dyson,NaN,...,Australian,individual,https://api.nga.gov/iiif/9b625788-593a-4e9a-9c...,0.000356,Australia,AU,Oceania,"""Ah_Mr._Hardy,_Mr._Hardy,_if_you_only_knew_all...",../../non_laImages,True
1,357,6827.0,"""Can you forgive me Father?"" - ""Can you forgiv...",drypoint,NaN,NaN,NaN,"Dyson, Will",Will Dyson,NaN,...,Australian,individual,https://api.nga.gov/iiif/1446efef-983e-4ea6-8c...,0.000356,Australia,AU,Oceania,"""Can_you_forgive_me_Father?""_-_""Can_you_forgiv...",../../non_laImages,True
2,116,172846.0,A Nicely Built City Never Resists Destruction,"etching, aquatint, and drypoint in black on wo...",plate: 29.53 × 37.78 cm (11 5/8 × 14 7/8 in.)\...,lower center in plate: A NICELY BUILT CITY NEV...,embossed lower left: CC,"Kentridge, William",William Kentridge,"(Robert Brown Gallery, Washington, DC); Corcor...",...,South African,individual,https://api.nga.gov/iiif/f857f34f-feda-4107-88...,0.000176,South Africa,ZA,Africa,A_Nicely_Built_City_Never_Resists_Destruction_...,../../non_laImages,True
3,31,133986.0,"Act I, Scene 2, from Ubu Tells the Truth","etching, aquatint, drypoint, and engraving on ...",image: 24.7 x 30 cm (9 3/4 x 11 13/16 in.)\nsh...,lower right in graphite: W Kentridge; lower le...,BS: CC,"Kentridge, William",William Kentridge,"(Robert Brown Gallery, Arlington, October 5, 2...",...,South African,individual,https://api.nga.gov/iiif/a88aa0ba-06c9-4bb5-8f...,0.000176,South Africa,ZA,Africa,"Act_I,_Scene_2,_from_Ubu_Tells_the_Truth_Kentr...",../../non_laImages,True
4,646,133985.0,"Act II, Scene 1, from Ubu Tells the Truth","etching, aquatint, drypoint, and engraving on ...",image: 24.9 x 30 cm (9 13/16 x 11 13/16 in.)\n...,lower right in graphite: W Kentridge; lower le...,BS: CC,"Kentridge, William",William Kentridge,"(Robert Brown Gallery, Arlington, October 5, 2...",...,South African,individual,https://api.nga.gov/iiif/7ad5f58f-f93b-4e7a-ad...,0.000176,South Africa,ZA,Africa,"Act_II,_Scene_1,_from_Ubu_Tells_the_Truth_Kent...",../../non_laImages,True


In [82]:
#this value is close enough to the 1207 files stored in the directory, and after downloading the files in 'to keep' the number should increase
nonLa_image_metadata.shape[0] - len(to_trash)

1275

In [83]:
nonLa_image_metadata = nonLa_image_metadata.drop(to_trash.tolist(), axis=0)

In [85]:
nonLa_image_metadata

,objectid,title,medium,dimensions,inscription,markings,attributioninverted,attribution,provenancetext,creditline,...,nationality,constituenttype,expanded_url,percent_fromCountry,Country Name,Country,Continent,file_name,directory,file_downloaded
2,48779.0,Tanned Hide,2-color lithograph,NaN,NaN,NaN,"Tamagami, Tsuneo",Tsuneo Tamagami,NaN,Rosenwald Collection,...,Japanese,individual,https://api.nga.gov/iiif/14c93846-ac93-4f1a-b6...,0.001668,Japan,JP,Asia,Tanned_Hide_Tamagami.jpg,../../non_laImages,True
3,72470.0,Modern Print,color lithograph and screenprint on Special Ar...,image: 61 x 61 cm (24 x 24 in.)\nsheet: 78.8 x...,lower left in graphite: Gemini III; lower righ...,embossed lower right: © [Gemini G.E.L.]; stamp...,"Lichtenstein, Roy",Roy Lichtenstein,"Gemini G.E.L., Los Angeles, California; acquir...",Gift of Gemini G.E.L. and the Artist,...,American,individual,https://api.nga.gov/iiif/ae4712b9-ca98-42c9-94...,0.709936,United States,US,North America,Modern_Print_Dressen.jpg,../../non_laImages,True
4,142127.0,Truman Capote,gelatin silver print,overall: 17.46 x 25.4 cm (6 7/8 x 10 in.),"on verso, by unknown hand, upper right in grap...",NaN,"Seymour, David",David Seymour (Chim),"Ben Shneiderman, Bethesda, MD; gift to NGA, 2008.",Gift of Ben Shneiderman,...,American,individual,https://api.nga.gov/iiif/24b4d1da-d11d-4c0d-a7...,0.709936,United States,US,North America,Truman_Capote_Seymour.jpg,../../non_laImages,True
5,4663.0,"The Gorbals, Glasgow",graphite,overall: 19 x 27.9 cm (7 1/2 x 11 in.),lower right in graphite: Muirhead Bone; in lat...,NaN,"Bone, Muirhead, Sir",Sir Muirhead Bone,LJR (Lugt Supp.1760b),Rosenwald Collection,...,American,individual,https://api.nga.gov/iiif/3b839fa8-b422-403a-b7...,0.709936,United States,US,North America,"The_Gorbals,_Glasgow_Rosenwald.jpg",../../non_laImages,True
7,3714.0,Christ before Herod,"woodcut, hand-colored in red lake, green, yell...",NaN,NaN,NaN,German 15th Century,German 15th Century,"Martin Aufhäuser [1875-1944], Munich, Holland,...",Rosenwald Collection,...,German,anonymous,https://api.nga.gov/iiif/27dac85b-217a-43f6-84...,0.040764,Germany,DE,Europe,Christ_before_Herod_German_15th_Century.jpg,../../non_laImages,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1944,30239.0,Ballads,bound volume with text and 5 etchings with eng...,page size: 16.2 × 9.1 cm (6 3/8 × 3 9/16 in.)\...,in gold tooled letters on spine of half red Mo...,NaN,"Hayley, William (author) and William Blake (ar...",William Hayley (author) and William Blake (art...,NaN,Gift of Lincoln Kirstein,...,British,individual,https://api.nga.gov/iiif/b54bd494-7811-40e2-ac...,0.029704,United Kingdom,GB,Europe,Ballads_Phillips.jpg,../../non_laImages,True
1946,42579.0,Et dire que voila trois... prévenu que je n'ai...,lithograph,NaN,NaN,NaN,"Daumier, Honoré",Honoré Daumier,NaN,Rosenwald Collection,...,French,individual,https://api.nga.gov/iiif/5c9c4566-2388-4e86-a4...,0.101900,France,FR,Europe,Et_dire_que_voila_trois..._prévenu_que_je_n'ai...,../../non_laImages,True
1948,52481.0,Auch ein Todtentanz: Text Page,typeset printing,NaN,NaN,NaN,"Rethel, Alfred",Alfred Rethel,NaN,Ailsa Mellon Bruce Fund,...,German,individual,https://api.nga.gov/iiif/32b7c514-f5a5-4b61-93...,0.040764,Germany,DE,Europe,Auch_ein_Todtentanz:_Text_Page_Rethel.jpg,../../non_laImages,True
1950,44709.0,Constancy Holding a Staff and Resting on a Col...,bronze,overall (diameter): 6.09 cm (2 3/8 in.)\ngross...,across bottom: CONSTANTIA,NaN,Cristoforo di Geremia,Cristoforo di Geremia,"Gustave Dreyfus [1837-1914], Paris; his heirs;...",Samuel H. Kress Collection,...,Italian,individual,https://api.nga.gov/iiif/b53ba8ca-41a9-48e7-be...,0.045247,Italy,IT,Europe,Constancy_Holding_a_Staff_and_Resting_on_a_Col...,../../non_laImages,True


### Redownloading the images with duplicate filenames that got overwritten to increase size of training and testing dataset in nonLa art (using objectID in new naming convention)

In [113]:
import shutil
import requests
def download_image(url, file_name, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(file_name, "wb") as f:
            f.write(response.content)
        shutil.move(file_name,'../../non_laImages/' + file_name)
    else:
        print(response.status_code)
#Define HTTP Headers
headers = {
    "User-Agent": "Chrome/51.0.2704.103",
}
import time
for i in range(0, len(to_keep)):
    #Define URL of an image
    url = to_keep.expanded_url[i]
    # Define image file name
    #check for nans in last name or title
    title = 'missing'
    last_name = 'missing'
    if isinstance(to_keep.title[i], str):
        title = to_keep.title[i]
    if isinstance(to_keep.lastname[i], str):
        last_name = to_keep.lastname[i]
    file_name = str.replace(title, ' ', '_').replace('/', '&') + '_' + str.replace(last_name, ' ', '_').replace('/', '&') + '_'  + str(int(to_keep.objectid[i])) + '.jpg'
    if len(file_name[:-4]) > 250:
        num_remove = len(file_name[:-4]) - 250
        file_name = file_name[:-(4 + num_remove)] + '.jpg'
    to_keep['file_name'][i] = file_name
    # Download image
    #timer delay (15 seconds)
    #time.sleep(15)
    #download_image(url, file_name, headers)

Dyson
"Ah Mr. Hardy, Mr. Hardy, if you only knew all the circumstances..."
Chinese Qing Dynasty
"Beehive" Water Pot
Chinese Qing Dynasty
"Beehive" Water Pot
Chinese Qing Dynasty
"Beehive" Water Pot
Chinese Qing Dynasty
"Beehive" Water Pot
Chinese Qing Dynasty
"Beehive" Water Pot
Dyson
"Can you forgive me Father?" - "Can you forgive me daughter?"
Kentridge
A Nicely Built City Never Resists Destruction
Kentridge
Act I, Scene 2, from Ubu Tells the Truth
Kentridge
Act II, Scene 1, from Ubu Tells the Truth
Kentridge
Act II, Scene 5, from Ubu Tells the Truth
Kentridge
Act III, Scene 4, from Ubu Tells the Truth
Kentridge
Act III, Scene 9, from Ubu Tells the Truth
Kentridge
Act IV, Scene 1, from Ubu Tells the Truth
Kentridge
Act IV, Scene 7, from Ubu Tells the Truth
Kentridge
Act V, Scene 4, from Ubu Tells the Truth
Chinese Qing Dynasty
Amphora Vase
Chinese Qing Dynasty
Amphora Vase
Chinese Qing Dynasty
Amphora Vase
Chinese Qing Dynasty
Amphora Vase
Chinese Qing Dynasty
Amphora Vase
Chinese Qi

### Renaming the rest of the files according to the new naming convention to individuate files. & removing the redundant columns from the metadata (These don't have duplicates)

In [86]:
nonLa_image_metadata = nonLa_image_metadata.where(nonLa_image_metadata.objectid.apply(pd.notnull)).dropna(how='all')

In [89]:
nonLa_image_metadata = nonLa_image_metadata.reset_index(drop=True)

In [90]:
import os

In [91]:
from os.path import exists

In [63]:
folder = '../../non_laImages/'
already_fixed = list(to_keep.objectid)
for i in range(0, len(nonLa_image_metadata)):
    # Renaming filenames in overall metadata to recognize new naming pattern =, only if not already in to_download
    if nonLa_image_metadata.objectid[i] not in already_fixed:
        title = 'missing'
        last_name = 'missing'
        if isinstance(nonLa_image_metadata.title[i], str):
            title = nonLa_image_metadata.title[i]
        if isinstance(nonLa_image_metadata.lastname[i], str):
            last_name = nonLa_image_metadata.lastname[i]
        old_file_name = nonLa_image_metadata.file_name[i]
        new_file_name = str.replace(title, ' ', '_').replace('/', '&') + '_' + str.replace(last_name, ' ', '_').replace('/', '&') + '_'  + str(int(nonLa_image_metadata.objectid[i])) + '.jpg'
        if len(new_file_name[:-4]) > 250:
            num_remove = len(new_file_name[:-4]) - 250
            new_file_name = new_file_name[:-(4 + num_remove)] + '.jpg'
        #renaming the file in the directory
        oldName = os.path.join(folder, old_file_name)
        newName = os.path.join(folder, new_file_name)

2
3
4
5
7
8
10
11
12
13
15
18
19
22
24
25
26
27
28
29
30
33
36
39
40
41
43
44
46
47
48
49
50
51
52
53
54
55
59
60
61
63
65
66
68
69
70
72
74
76
78
81
82
83
85
86
87
88
89
91
93
95
96
97
98
99
100
101
102
103
104
106
107
108
109
110
113
115
118
121
123
124
125
128
129
130
131
134
137
138
139
140
141
143
145
147
148
149
150
152
154
155
156
157
159
162
163
164
166
167
168
171
172
173
175
176
177
178
179
180
181
184
185
187
189
191
192
193
198
199
200
201
203
206
207
208
213
215
219
220
222
224
226
228
229
230
232
235
236
237
240
242
245
247
248
249
250
251
252
253
256
257
258
259
260
261
262
263
264
267
268
269
271
272
274
278
279
280
283
285
286
288
289
290
291
292
296
297
298
301
303
305
306
307
308
309
315
316
317
318
320
323
325
326
327
328
330
331
332
336
338
339
340
341
343
344
349
350
351
353
354


/Users/mocha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


355
360
363
365
366
371
372
373
375
376
377
378
379
380
382
384
385
388
390
392
393
395
397
400
401
404
406
407
410
412
415
419
420
421
425
426
427
428
432
437
440
443
444
445
447
448
449
450
451
452
453
454
455
460
467
468
470
471
479
480
481
484
486
487
489
490
494
495
497
499
501
502
503
504
505
506
507
508
509
511
512
520
521
525
527
530
532
533
534
535
536
537
538
539
544
545
546
548
549
553
554
556
557
560
564
565
568
569
575
576
577
578
579
581
583
587
588
590
592
593
594
596
597
598
599
600
601
602
603
606
609
611
612
614
616
617
618
621
622
623
624
625
626
627
628
629
630
631
632
640
641
642
645
648
649
651
652
654
655
657
659
660
661
662
663
664
668
669
670
673
674
676
677
679
680
681
683
684
685
687
691
694
695
698
700
701
703
705
706
708
710
711
712
715
716
717
720
722
723
728
729
730
733
735
738
743
745
747
748
749
752
753
754
755
757
760
761
763
765
766
768
769
770
773
774
775
777
780
782
783
786
787
789
790
791
792
797
798
802
803
804
807
809
810
813
814
815
816
818
819


### Deleting the files with the previous naming convention (if the new name didn't take effect)

In [92]:
folder = '../../non_laImages/'
for i in range(0, len(nonLa_image_metadata)):
    if pd.notna(nonLa_image_metadata.objectid[i]):
        title = 'missing'
        last_name = 'missing'
        if isinstance(nonLa_image_metadata.title[i], str):
            title = nonLa_image_metadata.title[i]
        if isinstance(nonLa_image_metadata.lastname[i], str):
            last_name = nonLa_image_metadata.lastname[i]
        old_file_name = nonLa_image_metadata.file_name[i]
        new_file_name = str.replace(title, ' ', '_').replace('/', '&') + '_' + str.replace(last_name, ' ', '_').replace('/', '&') + '_'  + str(int(nonLa_image_metadata.objectid[i])) + '.jpg'
        oldName = os.path.join(folder, old_file_name)
        newName = os.path.join(folder, new_file_name)
        #if both exists, delete old named file
        #if exists(oldName) and exists(newName):
            #delete oldName & update metadata with new name
        #    os.remove(oldName)
        #    nonLa_image_metadata['file_name'][i] = new_file_name
        #elif exists(oldName) and not exists(newName):
            # update to new naming convention and delete oldName and update metadata
        #    os.rename(oldName, newName)
            #renaming the file in the metadata
        #    nonLa_image_metadata['file_name'][i] = new_file_name
        if exists(newName):
            nonLa_image_metadata['file_name'][i] = new_file_name

/Users/mocha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [95]:
nonLa_image_metadata.file_name.head()

0                      Tanned_Hide_Tamagami_48779.jpg
1                      Modern_Print_Dressen_72470.jpg
2                    Truman_Capote_Seymour_142127.jpg
3             The_Gorbals,_Glasgow_Rosenwald_4663.jpg
4    Christ_before_Herod_German_15th_Century_3714.jpg
Name: file_name, dtype: object

### Reducing metadata to those files which were downloaded and indexed without a hitch

In [97]:
location_name = nonLa_image_metadata.directory + '/' + nonLa_image_metadata.file_name

In [99]:
location_name.head()

0    ../../non_laImages/Tanned_Hide_Tamagami_48779.jpg
1    ../../non_laImages/Modern_Print_Dressen_72470.jpg
2    ../../non_laImages/Truman_Capote_Seymour_14212...
3    ../../non_laImages/The_Gorbals,_Glasgow_Rosenw...
4    ../../non_laImages/Christ_before_Herod_German_...
dtype: object

In [100]:
nonLa_image_metadata['file_downloaded'] = location_name.apply(exists)

In [102]:
downloaded = nonLa_image_metadata.where(nonLa_image_metadata.file_downloaded).dropna(how='all')

In [105]:
missing = nonLa_image_metadata.where(nonLa_image_metadata.file_downloaded.apply(lambda x: not x)).dropna(how='all')

In [108]:
missing_filenames = missing.directory + '/' + missing.file_name

In [112]:
missing_filenames = missing_filenames.reset_index(drop=False)

### Next up-! Splitting the downloaded Non La Art, for training and testing (face detection), if there isn't enough data to do faces, will move on to style transfer

In [125]:
downloaded.to_csv('../../data_samples/results/whole_set_results/local_nonLa_art.csv', index=False)